In [51]:
from io import StringIO
import pandas as pd
import requests 
import datetime
from bs4 import BeautifulSoup
import time
from mongo import Mongo

In [33]:
url = 'https://mops.twse.com.tw/server-java/t164sb01'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
payload = {
                'step': 1,
                'CO_ID': str(2330),
                'SYEAR': str(2022),
                'SSEASON': str(1),
                'REPORT_ID': 'C'
            }
response = requests.post(url, headers=headers, data=payload)
response.encoding='big5'
dfs = pd.read_html(StringIO(response.text))
df =dfs[0]
df
data = df.reset_index()
#data.columns = data.columns.droplevel(0)
data
# season_data = data.iloc[:,3]
# season_data


# # Create a new row to append as a Series
# new_row = pd.Series([50], index=[0])

# # Append the new row to the original Series at the top
# updated_series = pd.concat([new_row, season_data], ignore_index=True)
# updated_series
# # data = data.reset_index().drop(columns='level_1')

# #data.columns = data.iloc[0]
# # data = data.drop(data.index[0])
# # data = data.rename(columns={'會計項目':'Timestamp'})
# # data = data.iloc[0]
# # new_timestamp = data['Timestamp'].replace('年第','Q').replace('季','')
# # data['Timestamp'] = new_timestamp
# #data = data.iloc[2]
# index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
# index_to_terms_dict

index 資產負債表Balance Sheet  \
                     代號Code   
0      0                NaN   
1      1                NaN   
2      2               1100   
3      3               1110   
4      4               1120   
..   ...                ...   
85    85               36XX   
86    86               3XXX   
87    87               3X2X   
88    88               3998   
89    89               3999   

                                                                           \
                                 會計項目Accounting Title 2022年3月31日2022/3/31   
0                                            資產Assets                 NaN   
1                                 流動資產　Current assets                 NaN   
2                   現金及約當現金 Cash and cash equivalents          1151589646   
3   透過損益按公允價值衡量之金融資產－流動 Current financial assets a...              758452   
4   透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial asse...           120995765   
..                                                ...                 ...   
85                    非控制權益 Non-controlling interests             7038658   
86                                  權益總額 Total equity          2321468754   
87               負債及權益總計　Total liabilities and equity          3992676667   
88  預收股款（權益項下）之約當發行股數　Equivalent issue shares of a...                   0   
89  母公司暨子公司所持有之母公司庫藏股股數（單位：股）　Number of shares in ...             1387000   

                                              
   2021年12月31日2021/12/31 2021年3月31日2021/3/31  
0                    NaN                 NaN  
1                    NaN                 NaN  
2             1064990192           664727857  
3                 159048               26897  
4              119519251           125208307  
..                   ...                 ...  
85               2446652             2060566  
86            2170733205          1940996962  
87            3725503455          2919131461  
88                     0                   0  
89                     0                   0  

[90 rows x 6 columns]

In [61]:
def process_df(df):
    data = df.reset_index()
    season_data = data.iloc[:,2]
    # # Create a new row to append as a Series
    new_row = pd.Series(['Timestamp'], index=[0])

    # # Append the new row to the original Series at the top
    updated_series = pd.concat([new_row, season_data], ignore_index=True)

    index_to_terms_dict = {str(index): term for index, term in enumerate(updated_series)}

    return index_to_terms_dict



def get_index_after_2019():
    indexs = []
    url = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2020&SSEASON=4&REPORT_ID=C'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    # payload = {
    #                 'step': 1,
    #                 'CO_ID': str(2330),
    #                 'SYEAR': str(2022),
    #                 'SSEASON': str(1),
    #                 'REPORT_ID': 'C'
    #             }
    response = requests.post(url, headers=headers)#, data=payload)
    response.encoding='big5'
    dfs = pd.read_html(StringIO(response.text))
    balance_df = dfs[0]
    income_df =dfs[1]
    cash_flow_df = dfs[2]

    balance_s = process_df(balance_df)
    income_s = process_df(income_df)
    cash_flow_s = process_df(cash_flow_df)
    return balance_s, income_s, cash_flow_s
    
balance_s, income_s, cash_flow_s = get_index_after_2019()
income_s
repo = Mongo(db='trading_bot', collection='income_sheet_index_after2019')
repo.send_document(income_s)
repo = Mongo(db='trading_bot', collection='cash_flow_index_after2019')
repo.send_document(cash_flow_s)
repo = Mongo(db='trading_bot', collection='balance_sheet_index_after2019')
repo.send_document(balance_s)

In [46]:
url = f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2020&SSEASON=4&REPORT_ID=C'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers)
response.encoding='big5'
dfs = pd.read_html(StringIO(response.text))
data = dfs[2]
data = data.reset_index()
data
season_data = data.iloc[:,3]
season_data[0] = 'a'
index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
index_to_terms_dict

/var/folders/ty/0sfpr_454fxfc8pv3h3387m00000gq/T/ipykernel_3897/796508371.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data[0] = 'a'


{0: 'a',
 1: '35353306',
 2: '35353306',
 3: nan,
 4: nan,
 5: '6844158',
 6: '420925',
 7: '(32,170)',
 8: '1986208',
 9: '(1,363,489)',
 10: '(1,567,644)',
 11: '4808',
 12: '(3,200,243)',
 13: '101742',
 14: '0',
 15: '101299',
 16: '(72,989)',
 17: '(135,851)',
 18: '5939',
 19: '3092693',
 20: nan,
 21: nan,
 22: '175844',
 23: '137727',
 24: '2284125',
 25: '(679,413)',
 26: '21721',
 27: '(50,258)',
 28: '385021',
 29: '320090',
 30: '(418,327)',
 31: '(186,551)',
 32: '1989979',
 33: nan,
 34: '(1,308,160)',
 35: '258701',
 36: '(389,063)',
 37: '76969',
 38: '(36,242)',
 39: '(1,397,795)',
 40: '592184',
 41: '3684877',
 42: '39038183',
 43: '(7,758,907)',
 44: '31279276',
 45: nan,
 46: '(34,101)',
 47: '1057782',
 48: '(26,198,078)',
 49: '(233,942)',
 50: '0',
 51: '(9,572,441)',
 52: '64452',
 53: '0',
 54: '(46,651)',
 55: '(1,085,531)',
 56: '2278219',
 57: '181983',
 58: '1174062',
 59: '2466331',
 60: '0',
 61: '(29,947,915)',
 62: nan,
 63: '4533158',
 64: '371631',
 

In [35]:
index_to_terms_dict = {index: term for index, term in enumerate(index_dict)}
cleaned_dict = {key: value.replace('\u3000', ' ') for key, value in index_to_terms_dict.items()}
cleaned_dict
from mongo import Mongo
repo = Mongo(db='trading_bot', collection='incomestatement_index')
repo.send_document(cleaned_dict)

documents must have only string keys, key was 0


In [121]:
# from mongo import Mongo

# financial_terms_dict = {term: index for index, term in enumerate(data.index)}
# financial_terms_dict
# repo = Mongo(db='trading_bot', collection='incomestatement_index')
# repo.send_document(financial_terms_dict)

In [67]:
new_index = []
for i in data.index:
    i = i.replace('年第','Q').replace('季','')
    new_index.append(i)


In [68]:
new_index

['2017Q3', '2016Q3', '2017年01月01日至2017年09月30日', '2016年01月01日至2016年09月30日']

In [14]:
res.encoding = 'big5'
dfs = pd.read_html(StringIO(res.text))
balance_sheet = dfs[1]
balance_sheet.columns = balance_sheet.columns.droplevel(1)

balance_sheet_index=balance_sheet.iloc[:, 0]
balance_sheet = balance_sheet.iloc[:, 1]
a = pd.concat([date_stock_id_value, balance_sheet])


NameError: name 'date_stock_id_value' is not defined

In [ ]:
date_stock_id_index = pd.Series(['Timestamp', 'stock_id'])

In [ ]:

import datetime as dt 
date = dt.datetime(2022, 2, 1)
date.quarter

In [ ]:
date_stock_id_value = pd.Series(['1101', pd.Timestamp('2021-09-30').strftime('%Y-%m-%d')])
b= pd.concat([date_stock_id_index, balance_sheet_index] )

In [ ]:
pd.concat([b,a], axis=1)

In [1]:
#!/usr/bin/env python
# coding: utf-8

import datetime
import random
import time
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None
class AllFinancialStatementsScraper:
    
    def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
        if start_year < 2013 or start_year < 0:
            raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
        if end_year < start_year or end_year < 0:
            raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
        if end_year > datetime.datetime.now().year:
            print('end_year 強制設定為今年')
            self.end_year = datetime.datetime.now().year
            self.year_now = self.end_year
        else:
            self.end_year = end_year
            self.year_now = datetime.datetime.now().year
        
        self.start_year = start_year
        self.income_statements_dfs = []
        self.balance_sheet_dfs = []
        self.cash_flow_dfs = []
        self.dfs = []
        
        self.year_range = list(range(self.start_year, self.end_year+1))
        self.season_range = list(range(1, 5))        
    
    # func 1 
    def get_all_statements_hst(self, stock_id=2330): # need to choose a stock id
        self.stock_id = stock_id

        # Calculate the newInfo_days for 財報發佈日 
        # func 2 is related
        mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
        mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
        may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
        aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
        nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        # Loop through each year and quarter to retrieve income statements
        for year in self.year_range:
            print(f'{year}年開爬!!')
            for season in self.season_range:                
                if year == self.end_year:
                    if (year == self.year_now - 1) & (season == 4):
                            if datetime.datetime.now() < mar_newInfo_day_B:
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
                                
                                return self.balance_sheet_dfs, self.income_statements_dfs, self.cash_flow_dfs
                    
                    elif year == self.year_now:
                        checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                        for i, newInfo_day in enumerate(checkTool):
                            if (datetime.datetime.now() < newInfo_day) & (season >= i):
                                print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')

                                return self.balance_sheet_dfs, self.income_statements_dfs, self.cash_flow_dfs
                ### main part ###    
                # func 3 is related
                self.scrape_all_statement(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
                ### main part ###
            
                if (year != self.start_year) & (season == 1):
                    time.sleep(random.uniform(4, 9))
                    print('每逢第一季，休息N秒')
                    
        print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
        self.dfs.append([self.income_statements_dfs,self.balance_sheet_dfs, self.cash_flow_dfs])
        return self.balance_sheet_dfs, self.income_statements_dfs, self.cash_flow_dfs
    
    # func 2
    def get_newInfo_day(self, y, m, d):
        newInfo_day = datetime.datetime(y, m, d)
        if newInfo_day.weekday() == 5:  # Saturday
            newInfo_day += datetime.timedelta(days=2)
        elif newInfo_day.weekday() == 6:  # Sunday
            newInfo_day += datetime.timedelta(days=1)
        return newInfo_day
    
    # func 3
    def scrape_all_statement(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
        while True:
            C_or_A = 'C'
            something_wrong = False

            url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

            url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
            payload = {
                'step': 1,
                'CO_ID': str(self.stock_id),
                'SYEAR': str(year),
                'SSEASON': str(season),
                'REPORT_ID': 'C'
            }

            try:
                response = requests.get(url, headers=headers)
                response.encoding='big5'
                tables = pd.read_html(StringIO(response.text))

                print(len(tables))
                print(tables)
                print(pd.read_html(str(tables)))
                'error 1'
                balance_sheet_df = tables[1]
                Incomestatement_df = tables[2]
                cash_flow_df = tables[3]
                #print(tables[2])
            except:
                try:
                    C_or_A = 'A'
                    res = requests.post(url_A, data=payload, headers=headers)
                    res.encoding = 'big5'
                    tables = pd.read_html(StringIO(res.text))
                    if year >= 2019:
                        
                        balance_sheet_df = tables[1]
                        Incomestatement_df = tables[2]
                        cash_flow_df = tables[3]

                        # #use pd to read the table
                        # Incomestatement_df = pd.read_html(str(Incomestatement_table))[0]
                        # balance_sheet_df = pd.read_html(str(balance_sheet_table))[0]
                        # cash_flow_df = pd.read_html(str(cash_flow_table))[0]
                    else:
                        print(len(tables))
                        print(tables)
                        print('error 3')
                        try:
                            balance_sheet_df = tables[0]
                        except Exception as e:
                            print(e)
                        Incomestatement_df = tables[1]
                        cash_flow_df = tables[2]

                        # #use pd to read the table
                        # Incomestatement_df = pd.read_html(str(Incomestatement_table))[0]
                        # balance_sheet_df = pd.read_html(str(balance_sheet_table))[0]
                        # cash_flow_df = pd.read_html(str(cash_flow_table))[0]
                    
                except Exception as e:
                    something_wrong = True
                    print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
                    print(f'於{year}Q{season}發生{e}')
                    time.sleep(16)

            if something_wrong == False:
                if C_or_A == 'C':
                    print('---這是合併財報---')
                    try:
                        #Incomestatement_df = pd.read_html(str(Incomestatement_table))[0]
                        if year >= 2019:
                            Incomestatement_data_dict = self.process_Incomestatement_df_after_2019(Incomestatement_df,year=year,season=season)

                        else:
                            Incomestatement_data_dict = self.process_Incomestatement_df_before_2019(Incomestatement_df,year=year,season=season)
                        self.income_statements_dfs.append([Incomestatement_data_dict])
                        #self.dfs.append(df)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
                        print(e)
                        time.sleep(16)

                     # Process the Balance Sheet data frame
                    try:
                        #balance_sheet_df = pd.read_html(str(balance_sheet_table))[0]
                        if year >= 2019:
                            balance_sheet_data_dict = self.process_balance_sheet_df_after_2019(balance_sheet_df, year=year, season=season)
                            print(balance_sheet_data_dict)
                        else:
                            balance_sheet_data_dict = self.process_balance_sheet_df_before_2019(balance_sheet_df, year=year, season=season)
                            print(balance_sheet_data_dict)

                        self.balance_sheet_dfs.append([balance_sheet_data_dict])
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'Balance Sheet finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'Balance Sheet: Error in {year}Q{season}')
                        print(e)
                        time.sleep(16)

                    # Process the Cash Flow data frame
                    try:
                        #cash_flow_df = pd.read_html(str(cash_flow_table))[0]
                        if year >= 2019:
                            cash_flow_data_dict = self.process_cash_flow_df_after_2019(cash_flow_df, year=year, season=season)
                        else:
                            cash_flow_data_dict = self.process_cash_flow_df_before_2019(cash_flow_df, year=year, season=season)

                        self.cash_flow_dfs.append([cash_flow_data_dict])
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'Cash Flow finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'Cash Flow: Error in {year}Q{season}')
                        print(e)
                        time.sleep(16)
                else:
                    print('---這是個別財報---')
                    try:
                        #Incomestatement_df = pd.read_html(str(Incomestatement_table))[0]
                        #income_statement_info_1season = [year, season, df]
                        if year >= 2019:
                            Incomestatement_data_dict = self.process_Incomestatement_df_after_2019(Incomestatement_df,year=year,season=season)
                        else:
                            Incomestatement_data_dict = self.process_Incomestatement_df_before_2019(Incomestatement_df,year=year,season=season)

                        self.income_statements_dfs.append([Incomestatement_data_dict])
                        #self.dfs.append(df)
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
                        time.sleep(16)
                         # Process the Balance Sheet data frame
                    try:
                        #balance_sheet_df = pd.read_html(str(balance_sheet_table))[0]
                        if year >= 2019:
                            balance_sheet_data_dict = self.process_balance_sheet_df_after_2019(balance_sheet_df, year=year, season=season)
                        else:
                            balance_sheet_data_dict = self.process_balance_sheet_df_before_2019(balance_sheet_df, year=year, season=season)

                        self.balance_sheet_dfs.append([balance_sheet_data_dict])
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'Balance Sheet finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'Balance Sheet: Error in {year}Q{season}')
                        time.sleep(16)

                    # Process the Cash Flow data frame
                    try:
                        #cash_flow_df = pd.read_html(str(cash_flow_table))[0]
                        if year >= 2019:
                            cash_flow_data_dict = self.process_cash_flow_df_after_2019(cash_flow_df, year=year, season=season)
                        else:
                            cash_flow_data_dict = self.process_cash_flow_df_before_2019(cash_flow_df, year=year, season=season)

                        self.cash_flow_dfs.append([cash_flow_data_dict])
                        print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'Cash Flow finished')
                    except Exception as e:
                        something_wrong = True
                        print(f'Cash Flow: Error in {year}Q{season}')
                        time.sleep(16)

            time.sleep(random.uniform(0.5, 1.5))

            if something_wrong == False:
                break
    
    def process_Incomestatement_df_before_2019(self, df,year,season):
        data = df.transpose()
        data.reset_index(inplace=True)
        season_data = data.iloc[1]

        index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
        index_to_terms_dict


        return index_to_terms_dict
    def process_Incomestatement_df_after_2019(self, df,year,season):
        data = df.transpose()
        data.reset_index(inplace=True)
        season_data = data.iloc[2]
        season_data[0] = f'{year}Q{season}'
        index_to_terms_dict = {index: term for index, term in enumerate(season_data)}

        return index_to_terms_dict
    def process_balance_sheet_df_before_2019(self, df, year, season):
        data = df.reset_index()
        data.columns = data.columns.droplevel(0)

        season_data = data.iloc[:,2]


        # Create a new row to append as a Series
        new_row = pd.Series([f'{year}Q{season}'], index=[0])

        # Append the new row to the original Series at the top
        updated_series = pd.concat([new_row, season_data], ignore_index=True)
        index_to_terms_dict = {index: term for index, term in enumerate(updated_series)}
        return index_to_terms_dict

    def process_balance_sheet_df_after_2019(self, df, year, season):
        # Placeholder implementation

        data = df.reset_index()
        season_data = data.iloc[:,3]
        season_data.iloc[0] = f'{year}Q{season}'
        index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
        return index_to_terms_dict

    def process_cash_flow_df_before_2019(self, df, year, season):
        # Implement data frame processing similar to the income statement
        # Placeholder implementation:
        data = df.reset_index()
        season_data = data.iloc[:,3]
        if season==1:
            season_data[0] = f'{year}Q1'
        elif season ==4:
            season_data[0] = f'entire_{year}'
        else:
            season_data[0] = f'{year}Q1-{year}Q{season}'
        index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
        return index_to_terms_dict

    def process_cash_flow_df_after_2019(self, df, year, season):
        # Placeholder implementation
        data = df.reset_index()
        season_data = data.iloc[:,3]
        if season==1:
            season_data[0] = f'{year}Q1'
        elif season ==4:
            season_data[0] = f'entire_{year}'
        else:
            season_data[0] = f'{year}Q1-{year}Q{season}'
        index_to_terms_dict = {index: term for index, term in enumerate(season_data)}
        return index_to_terms_dict                 


if __name__ == '__main__': 
   
    # example 3
    IS_scraper = AllFinancialStatementsScraper() #實際在抓歷史資料時，用默認值即可
    balance_sheet, income_sheet, cash_flow = IS_scraper.get_all_statements_hst(2330) # 3372 is a good example showing 2 kind of 財報（合併 or 個別）
    



2013年開爬!!
於2013Q1發生【錯誤1】:若於出現則自動休眠(16s)再重跑
於2013Q1發生name 'StringIO' is not defined
於2013Q1發生【錯誤1】:若於出現則自動休眠(16s)再重跑
於2013Q1發生name 'StringIO' is not defined
於2013Q1發生【錯誤1】:若於出現則自動休眠(16s)再重跑
於2013Q1發生name 'StringIO' is not defined


KeyboardInterrupt: 

In [47]:
balance_sheet[2]

[{0: '2013Q3'}]

In [ ]:
def send_to_mongo():
    for income_statement in df[0]:
        

In [ ]:
# #!/usr/bin/env python
# # coding: utf-8

# import datetime
# import random
# import time
# import urllib.request
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# class IncomeStatementsScraper:
    
#     def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
#         if start_year < 2013 or start_year < 0:
#             raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
#         if end_year < start_year or end_year < 0:
#             raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
#         if end_year > datetime.datetime.now().year:
#             print('end_year 強制設定為今年')
#             self.end_year = datetime.datetime.now().year
#             self.year_now = self.end_year
#         else:
#             self.end_year = end_year
#             self.year_now = datetime.datetime.now().year
        
#         self.start_year = start_year
#         self.dfs = []
        
#         self.year_range = list(range(self.start_year, self.end_year+1))
#         self.season_range = list(range(1, 5))        
    
#     # func 1 
#     def get_income_statements_hst(self, stock_id=2330): # need to choose a stock id
#         self.stock_id = stock_id

#         # Calculate the newInfo_days for 財報發佈日 
#         # func 2 is related
#         mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
#         mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
#         may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
#         aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
#         nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

#         headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#         # Loop through each year and quarter to retrieve income statements
#         for year in self.year_range:
#             print(f'{year}年開爬!!')
#             for season in self.season_range:                
#                 if year == self.end_year:
#                     if (year == self.year_now - 1) & (season == 4):
#                             if datetime.datetime.now() < mar_newInfo_day_B:
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 #self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
                    
#                     elif year == self.year_now:
#                         checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                         for i, newInfo_day in enumerate(checkTool):
#                             if (datetime.datetime.now() < newInfo_day) & (season >= i):
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 #self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
#                 ### main part ###    
#                 # func 3 is related
#                 self.scrape_income_statement(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                 ### main part ###
            
#                 if (year != self.start_year) & (season == 1):
#                     time.sleep(random.uniform(4, 9))
#                     print('每逢第一季，休息N秒')
                    
#         print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
#        # self.PK = self.PK_year_season(self.dfs) # func 6 is relatedff
#         return self.dfs
    
#     # func 2
#     def get_newInfo_day(self, y, m, d):
#         newInfo_day = datetime.datetime(y, m, d)
#         if newInfo_day.weekday() == 5:  # Saturday
#             newInfo_day += datetime.timedelta(days=2)
#         elif newInfo_day.weekday() == 6:  # Sunday
#             newInfo_day += datetime.timedelta(days=1)
#         return newInfo_day
    
#     # func 3
#     def scrape_income_statement(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
#         while True:
#             C_or_A = 'C'
#             something_wrong = False

#             url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

#             url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
#             payload = {
#                 'step': 1,
#                 'CO_ID': str(self.stock_id),
#                 'SYEAR': str(year),
#                 'SSEASON': str(season),
#                 'REPORT_ID': 'A'
#             }

#             try:
#                 req = urllib.request.Request(url, headers=headers)
#                 response = urllib.request.urlopen(req)
#                 html_content = response.read().decode('cp950')
#                 soup = BeautifulSoup(html_content, 'html.parser')
#                 tables = soup.find_all('table')

#                 ghost = tables[3]

#                 main_table = tables[1]
#             except:
#                 try:
#                     C_or_A = 'A'
#                     res = requests.post(url_A, data=payload, headers=headers)
#                     res.encoding = 'big5'
#                     soup = BeautifulSoup(res.text, 'html.parser')
#                     tables = soup.find_all('table')
#                     if year >= 2019:
#                         main_table = tables[1]
#                     else:
#                         main_table = tables[2]
                    
#                 except Exception as e:
#                     something_wrong = True
#                     print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
#                     print(f'於{year}Q{season}發生{e}')
#                     time.sleep(16)

#             if something_wrong == False:
#                 if C_or_A == 'C':
#                     print('---這是合併財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         if year >= 2019:
#                             data_dict = self.process_df_after_2019(df,year=year,season=season)
#                         else:
#                             data_dict = self.process_df_before_2019(df,year=year,season=season)
#                         self.dfs.append(data_dict)
#                         #self.dfs.append(df)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
#                         print(e)
#                         time.sleep(16)
#                 else:
#                     print('---這是個別財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         #income_statement_info_1season = [year, season, df]
#                         if year >= 2019:
#                             data_dict = self.process_df_after_2019(df,year=year,season=season)
#                         else:
#                             data_dict = self.process_df_before_2019(df,year=year,season=season)

#                         self.dfs.append(data_dict)
#                         #self.dfs.append(df)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
#                         time.sleep(16)

#             time.sleep(random.uniform(0.5, 1.5))

#             if something_wrong == False:
#                 break
    
#     def process_df_before_2019(self, df,year,season):
#         data = df.transpose()
#         data = data.reset_index().drop(columns='level_1')
#         data.columns = data.iloc[0]
#         data = data.drop(data.index[0])
#         data = data.rename(columns={'會計項目':'Timestamp'})
#         data = data.iloc[0]
#         new_timestamp = data['Timestamp'].replace('年第','Q').replace('季','')
#         data['Timestamp'] = new_timestamp
#         index_to_terms_dict = {index: term for index, term in enumerate(data)}
#         index_to_terms_dict[0] = f'{year}Q{season}'


#         return index_to_terms_dict
#     def process_df_after_2019(self, df,year,season):
#         data = df.transpose()
#         data.reset_index()
#         season_data = data.iloc[2]
#         season_data[0] = f'{year}Q{season}'
#         index_to_terms_dict = {index: term for index, term in enumerate(season_data)}

#         return index_to_terms_dict
                           
#     def get_stock_EPS(self, stock_income_statement):
#         # get feature EPS
#         EPS_list = []

#         for i, dfs in enumerate(stock_income_statement):
#             if dfs[0] <= 2018:
#                 Q_EPS = dfs[2][dfs[2].iloc[:, 0].isin(['基本每股盈餘合計', '基本每股盈餘合計　Total basic earnings per share'])].iloc[:, 1].values[0]

#                 if dfs[1] == 4:
#                     Q_EPS = Q_EPS - sum(EPS_list[-3:])
#                     EPS_list.append(Q_EPS)
#                 else:
#                     EPS_list.append(Q_EPS)
#             else:
#                 Q_EPS = dfs[2][dfs[2].iloc[:, 1].isin(['基本每股盈餘合計', '基本每股盈餘合計　Total basic earnings per share'])].iloc[:, 2].values[0]

#                 if type(Q_EPS) == str:
#                     if Q_EPS[0] == '(':
#                         Q_EPS = float(Q_EPS[1:-1]) * (-1)
#                     else:
#                         Q_EPS = float(Q_EPS)

#                 if dfs[1] == 4:
#                     Q_EPS = Q_EPS - sum(EPS_list[-3:])
#                     EPS_list.append(Q_EPS)
#                 else:
#                     EPS_list.append(Q_EPS)

#         df_EPS = pd.DataFrame({'單季EPS': EPS_list})
#         return df_EPS
    
#     # func 5
#     def get_rc_breaker(self, df, col_name='EPS創N季新高/低'):
#         # get feature __ record breaker (創N季新高/低)
#         N_ans = []

#         for i in range(len(df)):
#             if i > 0:
#                 value = df.iloc[i, 0]
#                 N = i + 1
#                 while N >= 2:
#                     if value == df.iloc[0:i + 1, 0].tail(N).max():
#                         N_ans.append(N)
#                         break
#                     elif value == df.iloc[0:i + 1, 0].tail(N).min():
#                         N_ans.append(-N)
#                         break
#                     else:
#                         N -= 1
#             else:
#                 N_ans.append(i)
#         df_rc_bk = pd.DataFrame({col_name: N_ans})

#         return df_rc_bk
#    # func 6
#     def PK_year_season(self, dfs):
#         y_list = [] # for year
#         s_list = [] # for season
        
#         for df in dfs:
#             y_list.append(str(df[0]))
#             s_list.append('Q' + str(df[1]))
        
#         df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
#         return df_PK_year_season

# if __name__ == '__main__': 
#     # example 1
#     # IS_scraper = IncomeStatementsScraper(start_year=2021)
#     # IS_3372 = IS_scraper.get_income_statements_hst(3372)
    
#     # example 2
#     # IS_scraper = IncomeStatementsScraper(end_year=2015)
#     # IS_2337 = IS_scraper.get_income_statements_hst(2337)
    
#     # example 3
#     IS_scraper = IncomeStatementsScraper() #實際在抓歷史資料時，用默認值即可
#     IS_2330 = IS_scraper.get_income_statements_hst(2330) # 3372 is a good example showing 2 kind of 財報（合併 or 個別）
#     # EPS_3372 = IS_scraper.get_stock_EPS(IS_3372)                                              # 特徵之一，形式為df
#     # EPS_rc_3372 = IS_scraper.get_rc_breaker(EPS_3372, '(自訂義名稱-創N期新高/低)')              # 特徵之二，形式為df
    
#     #final_gift = pd.concat([IS_scraper.PK,EPS_3372,EPS_rc_3372], axis=1) # 把主鍵IS_scraper.PK與我們要的特徵合併起來
#     # print('========================最終===============================')
#     # print(final_gift) 
#     # print('========================特徵1==============================')
#     # print(EPS_3372)
#     # print('========================特徵2==============================')
#     # print(EPS_rc_3372)





In [18]:
# #!/usr/bin/env python
# # coding: utf-8

# import datetime
# import random
# import time
# import urllib.request
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# class Cash_Flow_Scraper:
    
#     def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
#         if start_year < 2013 or start_year < 0:
#             raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
#         if end_year < start_year or end_year < 0:
#             raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
#         if end_year > datetime.datetime.now().year:
#             print('end_year 強制設定為今年')
#             self.end_year = datetime.datetime.now().year
#             self.year_now = self.end_year
#         else:
#             self.end_year = end_year
#             self.year_now = datetime.datetime.now().year
        
#         self.start_year = start_year
#         self.dfs = []
        
#         self.year_range = list(range(self.start_year, self.end_year+1))
#         self.season_range = list(range(1, 5))        
    
#     # func 1 
#     def get_cash_flow_hst(self, stock_id=2330): # need to choose a stock id
#         self.stock_id = stock_id

#         # Calculate the newInfo_days for 財報發佈日 
#         # func 2 is related
#         mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
#         mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
#         may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
#         aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
#         nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

#         headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#         # Loop through each year and quarter to retrieve income statements
#         for year in self.year_range:
#             print(f'{year}年開爬!!')
#             for season in self.season_range:                
#                 if year == self.end_year:
#                     if (year == self.year_now - 1) & (season == 4):
#                             if datetime.datetime.now() < mar_newInfo_day_B:
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
                    
#                     elif year == self.year_now:
#                         checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                         for i, newInfo_day in enumerate(checkTool):
#                             if (datetime.datetime.now() < newInfo_day) & (season >= i):
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
#                 ### main part ###    
#                 # func 3 is related
#                 self.scrape_cash_flow(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                 ### main part ###
            
#                 if (year != self.start_year) & (season == 1):
#                     time.sleep(random.uniform(4, 9))
#                     print('每逢第一季，休息N秒')
                    
#         print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
#         self.PK = self.PK_year_season(self.dfs) # func 6 is related
#         return self.dfs
    
#     # func 2
#     def get_newInfo_day(self, y, m, d):
#         newInfo_day = datetime.datetime(y, m, d)
#         if newInfo_day.weekday() == 5:  # Saturday
#             newInfo_day += datetime.timedelta(days=2)
#         elif newInfo_day.weekday() == 6:  # Sunday
#             newInfo_day += datetime.timedelta(days=1)
#         return newInfo_day
    
#     # func 3
#     def scrape_cash_flow(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
#         while True:
#             C_or_A = 'C'
#             something_wrong = False

#             url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

#             url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
#             payload = {
#                 'step': 1,
#                 'CO_ID': str(self.stock_id),
#                 'SYEAR': str(year),
#                 'SSEASON': str(season),
#                 'REPORT_ID': 'A'
#             }

#             try:
#                 req = urllib.request.Request(url, headers=headers)
#                 response = urllib.request.urlopen(req)
#                 html_content = response.read().decode('cp950')
#                 soup = BeautifulSoup(html_content, 'html.parser')
#                 tables = soup.find_all('table')

#                 ghost = tables[3]

#                 main_table = tables[2]
#             except:
#                 try:
#                     C_or_A = 'A'
#                     res = requests.post(url_A, data=payload, headers=headers)
#                     res.encoding = 'big5'
#                     soup = BeautifulSoup(res.text, 'html.parser')
#                     tables = soup.find_all('table')
#                     if year >= 2019:
#                         main_table = tables[2]
#                     else:
#                         main_table = tables[3]
#                 except:
#                     something_wrong = True
#                     print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
#                     time.sleep(16)

#             if something_wrong == False:
#                 if C_or_A == 'C':
#                     print('---這是合併財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         income_statement_info_1season = [year, season, df]
#                         self.dfs.append(income_statement_info_1season)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
#                         time.sleep(16)
#                 else:
#                     print('---這是個別財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         income_statement_info_1season = [year, season, df]
#                         self.dfs.append(income_statement_info_1season)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
#                         time.sleep(16)

#             time.sleep(random.uniform(0.5, 1.5))

#             if something_wrong == False:
#                 break
    
    
#     # func 6
#     def PK_year_season(self, dfs):
#         y_list = [] # for year
#         s_list = [] # for season
        
#         for df in dfs:
#             y_list.append(str(df[0]))
#             s_list.append('Q' + str(df[1]))
        
#         df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
#         return df_PK_year_season

# if __name__ == '__main__': 
   
    

#     BS_scraper = AllFinancialStatementsScraper()
#     result = 
        

AttributeError: 'AllFinancialStatementsScraper' object has no attribute 'get_balance_sheet_hst'

In [42]:
IS_3372

[]

In [12]:
# #!/usr/bin/env python
# # coding: utf-8

# import datetime
# import random
# import time
# import urllib.request
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# class Balance_Sheet_Scraper:
    
#     def __init__(self, start_year=2013, end_year=datetime.datetime.now().year):
#         if start_year < 2013 or start_year < 0:
#             raise ValueError("start_year 必須大於等於 2013 並且不能為負數")
#         if end_year < start_year or end_year < 0:
#             raise ValueError("end_year 必須大於等於 start_year 並且不能為負數")
#         if end_year > datetime.datetime.now().year:
#             print('end_year 強制設定為今年')
#             self.end_year = datetime.datetime.now().year
#             self.year_now = self.end_year
#         else:
#             self.end_year = end_year
#             self.year_now = datetime.datetime.now().year
        
#         self.start_year = start_year
#         self.dfs = []
        
#         self.year_range = list(range(self.start_year, self.end_year+1))
#         self.season_range = list(range(1, 5))        
    
#     # func 1 
#     def get_balance_sheet_hst(self, stock_id=2330): # need to choose a stock id
#         self.stock_id = stock_id

#         # Calculate the newInfo_days for 財報發佈日 
#         # func 2 is related
#         mar_newInfo_day_B = self.get_newInfo_day(1 + self.end_year, 3, 31)
#         mar_newInfo_day = self.get_newInfo_day(self.end_year, 3, 31)
#         may_newInfo_day = self.get_newInfo_day(self.end_year, 5, 15)
#         aug_newInfo_day = self.get_newInfo_day(self.end_year, 8, 14)
#         nov_newInfo_day = self.get_newInfo_day(self.end_year, 11, 14)

#         headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#         # Loop through each year and quarter to retrieve income statements
#         for year in self.year_range:
#             print(f'{year}年開爬!!')
#             for season in self.season_range:                
#                 if year == self.end_year:
#                     if (year == self.year_now - 1) & (season == 4):
#                             if datetime.datetime.now() < mar_newInfo_day_B:
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
                    
#                     elif year == self.year_now:
#                         checkTool = (mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                         for i, newInfo_day in enumerate(checkTool):
#                             if (datetime.datetime.now() < newInfo_day) & (season >= i):
#                                 print('已爬至歷史最新資料，正常結束。 產出主鍵：年、季度。')
#                                 self.PK = self.PK_year_season(self.dfs) # func 6 is related
#                                 return self.dfs
#                 ### main part ###    
#                 # func 3 is related
#                 self.scrape_balance_sheet(year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day)
#                 ### main part ###
            
#                 if (year != self.start_year) & (season == 1):
#                     time.sleep(random.uniform(4, 9))
#                     print('每逢第一季，休息N秒')
                    
#         print('正常結束，無錯誤。 產出主鍵：年、季度。')
        
#         self.PK = self.PK_year_season(self.dfs) # func 6 is related
#         return self.dfs
    
#     # func 2
#     def get_newInfo_day(self, y, m, d):
#         newInfo_day = datetime.datetime(y, m, d)
#         if newInfo_day.weekday() == 5:  # Saturday
#             newInfo_day += datetime.timedelta(days=2)
#         elif newInfo_day.weekday() == 6:  # Sunday
#             newInfo_day += datetime.timedelta(days=1)
#         return newInfo_day
    
#     # func 3
#     def scrape_balance_sheet(self, year, season, headers, mar_newInfo_day_B, mar_newInfo_day, may_newInfo_day, aug_newInfo_day, nov_newInfo_day):
#         while True:
#             C_or_A = 'C'
#             something_wrong = False

#             url = f'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID={self.stock_id}&SYEAR={year}&SSEASON={season}&REPORT_ID=C'

#             url_A = 'https://mops.twse.com.tw/server-java/t164sb01'
#             payload = {
#                 'step': 1,
#                 'CO_ID': str(self.stock_id),
#                 'SYEAR': str(year),
#                 'SSEASON': str(season),
#                 'REPORT_ID': 'A'
#             }

#             try:
#                 req = urllib.request.Request(url, headers=headers)
#                 response = urllib.request.urlopen(req)
#                 html_content = response.read().decode('cp950')
#                 soup = BeautifulSoup(html_content, 'html.parser')
#                 tables = soup.find_all('table')

#                 ghost = tables[3]

#                 main_table = tables[3]
#             except:
#                 try:
#                     C_or_A = 'A'
#                     res = requests.post(url_A, data=payload, headers=headers)
#                     res.encoding = 'big5'
#                     soup = BeautifulSoup(res.text, 'html.parser')
#                     tables = soup.find_all('table')
#                     if year >= 2019:
#                         main_table = tables[3]
#                     else:
#                         main_table = tables[4]
#                 except:
#                     something_wrong = True
#                     print(f'於{year}Q{season}發生【錯誤1】:若於出現則自動休眠(16s)再重跑')
#                     time.sleep(16)

#             if something_wrong == False:
#                 if C_or_A == 'C':
#                     print('---這是合併財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         income_statement_info_1season = [year, season, df]
#                         self.dfs.append(income_statement_info_1season)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤2】:若於出現則自動休眠(16s)再重跑')
#                         time.sleep(16)
#                 else:
#                     print('---這是個別財報---')
#                     try:
#                         df = pd.read_html(str(main_table))[0]
#                         income_statement_info_1season = [year, season, df]
#                         self.dfs.append(income_statement_info_1season)
#                         print('stock:' + str(self.stock_id), 'period:' + str(year), 'Q' + str(season), 'finished')
#                     except Exception as e:
#                         something_wrong = True
#                         print(f'於{year}Q{season}發生【錯誤3】:若於出現則自動休眠(16s)再重跑')
#                         time.sleep(16)

#             time.sleep(random.uniform(0.5, 1.5))

#             if something_wrong == False:
#                 break
    
    
#     # func 6
#     def PK_year_season(self, dfs):
#         y_list = [] # for year
#         s_list = [] # for season
        
#         for df in dfs:
#             y_list.append(str(df[0]))
#             s_list.append('Q' + str(df[1]))
        
#         df_PK_year_season = pd.DataFrame({'年度': y_list, '季度': s_list})
        
#         return df_PK_year_season

# if __name__ == '__main__': 
#    BS_scraper = Balance_Sheet_Scraper()
#    result = BS_scraper.get_balance_sheet_hst()

        

2013年開爬!!
---這是合併財報---
stock:2330 period:2013 Q1 finished
---這是合併財報---
stock:2330 period:2013 Q2 finished
---這是合併財報---
stock:2330 period:2013 Q3 finished
---這是合併財報---
stock:2330 period:2013 Q4 finished
2014年開爬!!
---這是合併財報---
stock:2330 period:2014 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2014 Q2 finished
---這是合併財報---
stock:2330 period:2014 Q3 finished
---這是合併財報---
stock:2330 period:2014 Q4 finished
2015年開爬!!
---這是合併財報---
stock:2330 period:2015 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2015 Q2 finished
---這是合併財報---
stock:2330 period:2015 Q3 finished
---這是合併財報---
stock:2330 period:2015 Q4 finished
2016年開爬!!
---這是合併財報---
stock:2330 period:2016 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2016 Q2 finished
---這是合併財報---
stock:2330 period:2016 Q3 finished
---這是合併財報---
stock:2330 period:2016 Q4 finished
2017年開爬!!
---這是合併財報---
stock:2330 period:2017 Q1 finished
每逢第一季，休息N秒
---這是合併財報---
stock:2330 period:2017 Q2 finished
---這是合併財報---
stock:2330 period:2017 Q3 fin

In [19]:
# from mongo import Mongo
# import datetime as dt
# class QuarterReportUpdater:
#     def __init__(self, stock_id, start_year=2013, end_year=dt.datetime.now().year):
#         # Initialize the scrapper
#         self.scrapper = IncomeStatementsScraper(start_year, end_year)
#         self.scrapper.stock_id = stock_id
        
#         # Initialize the MongoDB connection
#         self.mongo = Mongo(db='trading_bot', collection='quarter_report')
    
#     def update_latest_data(self):
#         # Get the latest data date from MongoDB
#         latest_date = self.mongo.get_latest_data_date()
        
#         # If there's no data in the database, start from the beginning
#         if not latest_date:
#             print("No data found in the database. Starting from the beginning.")
#             start_date = datetime(self.scrapper.start_year, 1, 1)
#         else:
#             # Else start from the next quarter after the latest date
#             next_quarter_month = (latest_date.month - 1) // 3 * 3 + 4
#             if next_quarter_month <= 12:
#                 start_date = datetime(latest_date.year, next_quarter_month, 1)
#             else:
#                 start_date = datetime(latest_date.year + 1, 1, 1)
                
#         # Get the end date
#         end_date = datetime(self.scrapper.end_year, 12, 31)
        
#         # Scrape data for the given date range
#         date_range = pd.date_range(start=start_date, end=end_date, freq='3M')
#         for date in date_range:
#             print(f"Scraping data for: {date}")
#             balance_sheet = self.scrapper.get_balance_sheet(date)
#             income_sheet = self.scrapper.get_income_sheet(date)
#             cash_flow_sheet = self.scrapper.get_cash_flow_sheet(date)
            
#             # Construct the record and send to MongoDB
#             record = {
#                 'Timestamp': date,
#                 'stock_id': self.scrapper.stock_id,
#                 'balance_sheet': balance_sheet.to_dict(),
#                 'income_sheet': income_sheet.to_dict(),
#                 'cash_flow_sheet': cash_flow_sheet.to_dict()
#             }
#             self.mongo.send_document(record)
#         print("Data update completed!")

In [ ]:
stock_id_list = Mongo(db='trading_bot',collection='quarter_report').get_stock_id_list()
stock_id_list

In [22]:
for id in stock_id_list:
    IncomeStatementUpdater(id)